# Worflow to train the ML model
#### Steps:
- crystal 
 - bulk input
- pymatgen
 - build molecule pymatgen (already oriented)
 - build slab
 - build slab + molecule 
- expert system
 - write crystal input
- optegom
- get adsorption energy
- post processing
 - charge transfer
 - band structure (or electronic properties?)
 - WF
- store data in reusable fashion?
- error management 

#### Loop:
- select list of substrates & adsorbates
- generate all surfaces
- optgeom substrates
- add adsorbates
- fragment optgeom
- extract info:
 - ads energy
 - charge transfer
 - electronic properties (band gap, work function, ?)


In [33]:
import sys
sys.path.insert(1, '../../crystal-code-tools/crystal-functions')

from cry_file_readwrite import write_cry_input, write_cry_gui
from cry_file_readwrite import Crystal_input, Crystal_output

from cry_adsorb import sub_ads_indices

from cry_calculate import cry_ads_energy

from cry_execute import runcry

from pymatgen.core.structure import Molecule, Structure, Lattice

from pymatgen.core.surface import SlabGenerator, generate_all_slabs

from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

from pymatgen.analysis.adsorption import AdsorbateSiteFinder

from pymatgen.ext.matproj import MPRester

import shutil
import pandas as pd
import numpy as np

### Make molecule 

In [71]:
co = Molecule('CO',[[0.0, 0.0, 0.0],[ 0.0, 0.0, 1.128]])
o = Molecule('O',[[0.0, 0.0, 0.0]])
adsorbate = o

### Make substrate

#### Bulk

In [3]:
with MPRester("p5vAQV3F1QuxFcxVT") as m:
    rutile = m.get_structure_by_material_id("mp-2657")
    anatase = m.get_structure_by_material_id("mp-390")
    cu = m.get_structure_by_material_id("mp-30")
    mgo = m.get_structure_by_material_id("mp-1265")

In [4]:
bulk = mgo
bulk = SpacegroupAnalyzer(bulk).get_conventional_standard_structure()

#### Slab

In [5]:
substrate = SlabGenerator(bulk, (1,0,0), 2., 10., center_slab=True).get_slab()

### Adsorb

In [73]:
sub_ads_structures = AdsorbateSiteFinder(substrate).adsorb_both_surfaces(adsorbate,repeat=[1,1,1])
AdsorbateSiteFinder(substrate).find_adsorption_sites()

sub_composition = []
ads_composition = []
miller_indices = []
n_layers = []
for i,sub_ads_structure in enumerate(sub_ads_structures):
    miller_indices.append(sub_ads_structure.miller_index)
    n_layers.append(len(np.unique(substrate.cart_coords[:,2])))  
    sub_composition.append(substrate.composition.reduced_formula)
    ads_composition.append(str(adsorbate.composition.hill_formula).replace(" ", ""))

### Prepare inputs (geometry optimisation)

In [7]:
#geom_block = ['Adsorption tests\n','EXTERNAL\n','EXTPRT\n','OPTGEOM\n','END\n']
geom_block = ['Adsorption tests\n','EXTERNAL\n','EXTPRT\n']
#bs_block = ['BASISSET\n', 'POB-DZVP\n']
bs_block = ['BASISSET\n', 'STO-3G\n']
func_block = ['DFT\n', 'B3LYP\n', 'XXLGRID\n', 'ENDDFT\n']
scf_block = [['TOLINTEG\n', '7 7 7 7 14\n'],
             ['SHRINK\n', '12 24\n'],
             ['MAXCYCLE\n', '200\n'],
             ['FMIXING\n', '70\n'],
             'DIIS\n',
             'ENDSCF\n']

#### Write inputs

In [8]:
file_names = []
for i,structure in enumerate(sub_ads_structures):
    input_name = 'data/'+str(substrate.composition).replace(" ", "")+'_'+str(adsorbate.composition).replace(" ", "")+'_' \
               +''.join(str(x) for x in substrate.miller_index)+'_'+str(i+1)+'.d12'
    file_names.append(input_name[:-4])
    write_cry_input(input_name,crystal_blocks=[geom_block,bs_block,func_block,scf_block],external_obj=structure)

### Run the calculations

In [9]:
for cry_input in file_names:
    #runcry(cry_input)
    pass

### Read the optimised geometry

In [16]:
E_full_system = []
for cry_input in file_names:
    cry_output = Crystal_output(cry_input+'.out')
    if cry_output.converged == True:
        E_full_system.append(cry_output.final_energy()) #Do I need this?

### Prepare inputs for the BSSE calculations

In [11]:
for i,cry_input in enumerate(file_names):
    if opt_structure[i] != None:
        indices = sub_ads_indices(sub_ads_structures[i])
        #Substrate        
        bsse_sub_inp = Crystal_input(cry_input+'.d12')
        bsse_sub_inp.add_ghost(indices['adsorbate'])
        bsse_sub_inp.opt_to_sp()
        bsse_sub_inp_name = cry_input+'_BSSE_sub.d12'
        write_cry_input(bsse_sub_inp_name,bsse_sub_inp)
        
        shutil.copy(cry_input+'.gui',cry_input+'_BSSE_sub.gui')
        
        #Adsorbate
        bsse_ads_inp = Crystal_input(cry_input+'.d12')
        bsse_ads_inp.add_ghost(indices['substrate'])
        bsse_sub_inp.opt_to_sp()
        bsse_ads_inp_name = cry_input+'_BSSE_ads.d12'
        write_cry_input(bsse_ads_inp_name,bsse_ads_inp)
        
        shutil.copy(cry_input+'.gui',cry_input+'_BSSE_ads.gui')

### Run the BSSE calculations

In [12]:
E_sub_BSSE = []
E_ads_BSSE = []
for i,cry_input in enumerate(file_names):
    runcry(cry_input+'_BSSE_sub')
    cry_BSSE_sub_output = Crystal_output(cry_input+'_BSSE_sub.out')
    if cry_BSSE_sub_output.converged == True:
        E_sub_BSSE.append(cry_BSSE_sub_output.final_energy())
    
    
    runcry(cry_input+'_BSSE_ads')
    cry_BSSE_ads_output = Crystal_output(cry_input+'_BSSE_ads.out')
    if cry_BSSE_ads_output.converged == True:
        E_ads_BSSE.append(cry_BSSE_ads_output.final_energy())

### Calculate the adsorption energy

In [76]:
E_adsorption = []
for i in range(len(file_names)):
    E_adsorption.append(cry_ads_energy(E_full_system[i],E_sub_BSSE[i],E_ads_BSSE[i]))

### Create Dataframe

In [75]:
df = pd.DataFrame(list(zip(file_names, sub_composition, n_layers, ads_composition,  miller_indices, E_adsorption,E_sub_BSSE,E_ads_BSSE,E_full_system)),
               columns =['File name', 'Substrate','N layers','Adsorbate','Miller Indices','E adsorption (BSSE)','E sub (ads ghost)','E ads (ads ghost)','E full system'])
df

,File name,Substrate,N layers,Adsorbate,Miller Indices,E adsorption (BSSE),E sub (ads ghost),E ads (ads ghost),E full system
0,data/Mg2O2_O1_100_1,MgO,2,O,"(1, 0, 0)",-2.340839,-14796.346942,-4022.516863,-18821.204644
1,data/Mg2O2_O1_100_2,MgO,2,O,"(1, 0, 0)",-3.669925,-14796.743457,-4021.823416,-18822.236798
2,data/Mg2O2_O1_100_3,MgO,2,O,"(1, 0, 0)",-1.953534,-14796.447112,-4022.175956,-18820.576601
3,data/Mg2O2_O1_100_4,MgO,2,O,"(1, 0, 0)",-3.961497,-14796.502362,-4022.206221,-18822.670080
